In [44]:
import os.path
import re

from typing import List

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [45]:
# html = open('../data/pdfs/ROCNE SPRAVY  2021/ACROSS FUNDING správa 2021.xhtml').read()
html = open('../../../data/pdfs/ROCNE SPRAVY  2021/OSIVO sprava 2021.xhtml').read()
# html = open('../data/pdfs/ROCNE SPRAVY  2021/Eurovea 2 financing sprava 2021.xhtml').read()
soup = BeautifulSoup(html)
[s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title', 'img'])]
# all_content = soup.find('div', attrs={'id': "page-container"})
visible_text = soup.getText()
# print(visible_text)
print(len(soup.findAll('p')))


1391


In [31]:
from bs4 import NavigableString, Tag


def most_frequent(arr):
    dict = {}
    count, itm = 0, ''
    for item in reversed(arr):
        dict[item] = dict.get(item, 0) + 1
        if dict[item] >= count :
            count, itm = dict[item], item
    return(itm)

def get_class_meta_from_div(found_classes: List, class_meta_name: str):
    # print(tag['class'])
    # print(class_meta_name)
    ret_list = [class_meta.split(class_meta_name)[1] for class_meta in found_classes if class_meta_name in class_meta]
    return ret_list if len(ret_list)>0 else [-1]

texts = {}
class_meta_names = ["ce1","ce2","ce3","ce4"]
text_count = -1
br_cnt = 0

for j, paragraph in enumerate(soup.findAll('td')):

    paragraph_text = paragraph.getText().strip()

    if not paragraph_text or all(not x.isalpha() for x in paragraph_text):
        continue

    classes = paragraph['class']
    meta_info = {}

    if paragraph.find('span'):
        other_classes = []
        for j, span in enumerate(paragraph.findAll('span')):

            span_text = span.getText().strip()

            if not span.has_attr('class') or \
                    not span_text or \
                    all(not x.isalpha() for x in span_text):
                continue
            classes += span['class']

            meta_info = {}
            other_classes = [cls for cls in classes if not cls.startswith(tuple(class_meta_names))]

            for class_meta_name in class_meta_names:
                meta_info[class_meta_name] = get_class_meta_from_div(classes, class_meta_name)

    else:
        other_classes = [cls for cls in classes if not cls.startswith(tuple(class_meta_names))]
        for class_meta_name in class_meta_names:
            meta_info[class_meta_name] = get_class_meta_from_div(classes, class_meta_name)

    text_blocks = []
    if paragraph.find('br'):
        for br in paragraph.findAll('br'):
            next_s = br.nextSibling
            if not (next_s and isinstance(next_s,NavigableString)):
                continue
            next2_s = next_s.nextSibling
            if next2_s and isinstance(next2_s,Tag) and next2_s.name == 'br':
                text = str(next_s).strip()
                if text:
                    text_blocks.append(next_s)
    else:
        text_blocks.append(paragraph_text)

    for extracted_text in text_blocks:
        text_count+=1
        texts[text_count]={}
        texts[text_count]["text"]=extracted_text
        texts[text_count]['page'] = 1
        texts[text_count]["other_classes"] = list(set(other_classes))
        for class_meta_name in class_meta_names:
            if class_meta_name in meta_info.keys():
                texts[text_count][class_meta_name] =  most_frequent(meta_info[class_meta_name])
            else:
                texts[text_count][class_meta_name] = -1



doc_df = pd.DataFrame.from_dict(texts, orient='index')

display(doc_df)


,text,page,other_classes,ce1,ce2,ce3,ce4
0,regulovaná informácia,1,[],,-1,-1,-1
1,ROČNÁ SPRÁVA,1,[],44,-1,-1,-1
2,emitenta,1,[],-1,,-1,-1
3,v zmysle zákona o burze cenných papierov,1,[],-1,,-1,-1
4,Časť 1.- Identifikácia emitenta,1,[],-1,-1,-1,
...,...,...,...,...,...,...,...
606,Test vyhovel formálnej kontrole,1,[ce656],-1,-1,-1,-1
607,Údaj o dlhopisoch,1,[ce649],-1,-1,-1,-1
608,Test vyhovel formálnej kontrole,1,[ce656],-1,-1,-1,-1
609,Vyhlásenie zodpovedných osôb emitenta,1,[ce649],-1,-1,-1,-1


In [32]:
for text in doc_df['text']:
    print(text)
    print("---------------")

regulovaná informácia
---------------
ROČNÁ SPRÁVA
---------------
emitenta
---------------
v zmysle zákona o burze cenných papierov
---------------
Časť 1.- Identifikácia emitenta
---------------
Informačná povinnosť za rok:
---------------
IČO:
---------------
LEI:
---------------
097900BGMX0000063288
---------------
Účtovné obdobie:
---------------
od:
---------------
do:
---------------
Právna forma
---------------
Akciová spoločnosť
---------------
Obchodné meno / názov:
---------------
OSIVO a.s.
---------------
Sídlo:
---------------
ulica, číslo
---------------
Kalinčiakova 2391
---------------
PSČ
---------------
Obec
---------------
Zvolen
---------------
Tel.:
---------------
smerové číslo
---------------
číslo:
---------------
Fax:
---------------
smerové číslo
---------------
číslo:
---------------
Webové sídlo:
---------------
www.osivo.sk
---------------
Dátum vzniku:
---------------
Základné imanie (v EUR):
---------------
Zakladateľ:
---------------
Oznámenie spôsobu z

In [33]:
def get_number_of_words(row):
    return len(row['text'].split())

def get_number_of_chars(row):
    return len(row['text'])

def get_number_of_alphachars(row):
    return sum(c.isalpha() for c in row['text'])

def store_text_info_only(dataframe_elements, out_firm_dir):
    # mask = (dataframe_elements.label == 'text')
    # dataframe_texts = dataframe_elements.copy()
    dataframe_elements['words'] = dataframe_elements.apply(get_number_of_words, axis=1)
    dataframe_elements['chars'] = dataframe_elements.apply(get_number_of_chars, axis=1)
    dataframe_elements['alpha'] = dataframe_elements.apply(get_number_of_alphachars, axis=1)
    dataframe_elements.to_csv(os.path.join(out_firm_dir, "all_texts_info.csv"))

    return dataframe_elements, os.path.join(out_firm_dir, "all_texts_info.csv")

dataframe_elements, dataframe_texts_csv_path = store_text_info_only(doc_df, "../../../data/out/")

In [34]:
def get_param_stats(dataframe_elements, param_name):
    dataframe_elements['index1'] = dataframe_elements.index
    # for param_val in dataframe_elements[param_name]:
        # print(param_val)
    # print([dataframe_elements[dataframe_elements[param_name] in param_val] for param_val in dataframe_elements[param_name] if param_val is not None ])
    param_counts = [ dataframe_elements[param_val == dataframe_elements[param_name]]["index1"].count() for param_val in
              dataframe_elements[param_name]]
    param_mean_word_count = [ dataframe_elements[param_val == dataframe_elements[param_name]]["words"].mean() for param_val in
              dataframe_elements[param_name]]
    # print(param_mean_word_count)
    sum_of_all_texts = dataframe_elements["index1"].count()
    dataframe_elements[f'{param_name}_frequency'] = param_counts / sum_of_all_texts
    dataframe_elements[f'{param_name}_mean_word_count'] = param_mean_word_count
    return dataframe_elements

dataframe_elements = get_param_stats(dataframe_elements, 'ce1')
dataframe_elements = get_param_stats(dataframe_elements, 'ce2')
dataframe_elements = get_param_stats(dataframe_elements, 'ce3')
dataframe_elements = get_param_stats(dataframe_elements, 'ce4')

display(dataframe_elements)
doc_df.to_csv("../data/out/test.csv")

# def store_base_file_stat(dataframe_elements, out_firm_dir):
#     dataframe_elements['index1'] = dataframe_elements.index
#     fs_counts = [dataframe_elements[dataframe_elements['fs'] == size]["index1"].count() for size in
#               dataframe_elements['fs']]
#     sum_of_all_texts = dataframe_elements["index1"].count()
#     dataframe_elements['size_frequency'] = counts / sum_of_all_texts
#     dataframe_elements.to_csv(os.path.join(out_firm_dir, "base_file_stat.csv"), index=False)
#
#     return dataframe_elements, os.path.join(out_firm_dir, "base_file_stat.csv")

,text,page,other_classes,ce1,ce2,ce3,ce4,words,chars,alpha,index1,ce1_frequency,ce1_mean_word_count,ce2_frequency,ce2_mean_word_count,ce3_frequency,ce3_mean_word_count,ce4_frequency,ce4_mean_word_count
0,regulovaná informácia,1,[],,-1,-1,-1,2,21,20,0,0.001637,2.000000,0.944354,10.736568,0.934534,10.674256,0.937807,10.752182
1,ROČNÁ SPRÁVA,1,[],44,-1,-1,-1,2,12,11,1,0.001637,2.000000,0.944354,10.736568,0.934534,10.674256,0.937807,10.752182
2,emitenta,1,[],-1,,-1,-1,1,8,8,2,0.914894,9.094812,0.003273,4.000000,0.934534,10.674256,0.937807,10.752182
3,v zmysle zákona o burze cenných papierov,1,[],-1,,-1,-1,7,40,34,3,0.914894,9.094812,0.003273,4.000000,0.934534,10.674256,0.937807,10.752182
4,Časť 1.- Identifikácia emitenta,1,[],-1,-1,-1,,4,31,25,4,0.914894,9.094812,0.944354,10.736568,0.934534,10.674256,0.003273,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,Test vyhovel formálnej kontrole,1,[ce656],-1,-1,-1,-1,4,31,28,606,0.914894,9.094812,0.944354,10.736568,0.934534,10.674256,0.937807,10.752182
607,Údaj o dlhopisoch,1,[ce649],-1,-1,-1,-1,3,17,15,607,0.914894,9.094812,0.944354,10.736568,0.934534,10.674256,0.937807,10.752182
608,Test vyhovel formálnej kontrole,1,[ce656],-1,-1,-1,-1,4,31,28,608,0.914894,9.094812,0.944354,10.736568,0.934534,10.674256,0.937807,10.752182
609,Vyhlásenie zodpovedných osôb emitenta,1,[ce649],-1,-1,-1,-1,4,37,34,609,0.914894,9.094812,0.944354,10.736568,0.934534,10.674256,0.937807,10.752182


FileNotFoundError: [Errno 2] No such file or directory: '../data/out/test.csv'

In [35]:
def store_unique_formatting_stats(dataframe_elements, out_firm_dir):
    dataframe_unique_formatting = dataframe_elements.groupby([ 'ce1', 'ce2', 'ce3', 'ce4', 'ce1_frequency', 'ce2_frequency', 'ce3_frequency', 'ce4_frequency',  'ce1_mean_word_count', 'ce2_mean_word_count', 'ce3_mean_word_count', 'ce4_mean_word_count']).agg({
        'text': list,
        "page": list,
        'index1': [list, 'count']})
    dataframe_unique_formatting.columns = ['_'.join(col).strip() for col in
                                           dataframe_unique_formatting.columns.values]
    dataframe_unique_formatting.reset_index(inplace=True)
    dataframe_unique_formatting.to_csv(os.path.join(out_firm_dir, "unique_formatting.csv"), index=True)

    return dataframe_unique_formatting, os.path.join(out_firm_dir, "unique_formatting.csv")

In [36]:
dataframe_unique_formatting, dataframe_unique_formatting_csv_path = store_unique_formatting_stats(dataframe_elements, "../data/out/")

FileNotFoundError: [Errno 2] No such file or directory: '../data/out/unique_formatting.csv'

In [40]:
def give_label(row):
    if row['ce1_frequency'] == 0 or row['ce2_frequency'] == 0 or row['ce3_frequency'] == 0 or row['ce4_frequency'] == 0:
        return "other"
    elif row['ce1_mean_word_count'] > 7 or row['ce2_mean_word_count'] > 7 or row['ce3_mean_word_count'] > 7 or row['ce4_mean_word_count'] > 7 :
        return "text"
    elif row['ce1_frequency'] > 0.01 or row['ce2_frequency'] > 0.01 or row['ce3_frequency'] > 0.01 or row['ce4_frequency'] > 0.01:
        return "header"
    else:
        return "other"
    # zvysok by mal byt trash -- nadpisy v tabulkach, chyby pri analyze pdf ...
    # return None

dataframe_unique_formatting['label'] = dataframe_unique_formatting.apply(give_label, axis=1)
dataframe_unique_formatting.to_csv(dataframe_unique_formatting_csv_path, index=True)
doc_df['label'] = doc_df.apply(give_label, axis=1)
doc_df.to_csv("../data/out/labelled.csv")

display(doc_df)

NameError: name 'dataframe_unique_formatting' is not defined

In [41]:
last_header = ''
headers = []
for text, label in zip(doc_df.text, doc_df.label):
    if label == 'header':
        last_header = text
    headers.append(last_header)
doc_df['belonging_header'] = headers
doc_df.to_csv("../data/out/assingned_headers.csv")

AttributeError: 'DataFrame' object has no attribute 'label'

In [39]:
def filter_texts(dataframe_texts, out_firm_dir):
    mask2 = (dataframe_texts.alpha > 40) & (dataframe_texts.words > 3)
    dataframe_texts_clean = dataframe_texts[mask2].copy()
    dataframe_texts_clean['d'] = 0
    dataframe_texts_clean['h'] = 0
    dataframe_texts_clean.to_csv(os.path.join(out_firm_dir, "filtered_texts_info.csv"))
    return dataframe_texts_clean, os.path.join(out_firm_dir, "filtered_texts_info.csv")
dataframe_texts_clean, dataframe_texts_clean_csv_path = filter_texts(doc_df, "../data/out/")

FileNotFoundError: [Errno 2] No such file or directory: '../data/out/filtered_texts_info.csv'